# Espacio Público

# Nuestro trabajo

In [13]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def create_driver(headless=False):
    """Crea y devuelve una instancia del driver de Chrome."""
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")  # Recomendado en modo headless
    service = Service()  # Se usa `Service` para evitar advertencias
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Crear el driver
driver = create_driver(headless=False)

# URL de la página
url = "https://espaciopublico.cl/nuestro-trabajo/"

# Abrir la página
driver.get(url)

# Esperar que cargue la página completamente
time.sleep(2)

# Hacer scroll hasta el final de la página para cargar más contenido
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Esperar la carga de contenido nuevo
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Obtener el HTML después de cargar el contenido
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encontrar todos los contenedores de proyectos
proyectos = soup.find_all('div', class_='proyecto-loop')

# Lista para almacenar los datos
datos = []

# Iterar sobre cada proyecto
for proyecto in proyectos:
    # Extraer título y enlace desde el segundo <a>
    enlaces = proyecto.find_all('a')
    if len(enlaces) > 1:  # Verificamos que haya al menos dos enlaces
        enlace_tag = enlaces[1]
        titulo_tag = enlace_tag.find('h3')
        titulo = titulo_tag.text.strip() if titulo_tag else "No title"
        enlace = enlace_tag['href']
    else:
        titulo = "No title"
        enlace = "No link"

    # Extraer la fecha
    fecha_tag = proyecto.find('span', class_='proyecto-loop-fecha')
    fecha = fecha_tag.text.strip() if fecha_tag else "No date"

    # Extraer las áreas de estudio
    areas_tag = proyecto.find('span', class_='proyecto-loop-area')
    areas = [a.text.strip() for a in areas_tag.find_all('a')] if areas_tag else ["No areas"]

    # Guardar los datos en un diccionario
    datos.append({
        "Título": titulo,
        "Enlace": enlace,
        "Fecha": fecha,
        "Áreas de estudio": ", ".join(areas)
    })

# Cerrar el navegador
driver.quit()

# Convertir la lista en un DataFrame
df = pd.DataFrame(datos)

# Mostrar el DataFrame
df

# Guardar en CSV (opcional)
# df.to_csv("articulos_espaciopublico.csv", index=False)

,Título,Enlace,Fecha,Áreas de estudio
0,No title,https://espaciopublico.cl/areas_de_estudio/seg...,30 de noviembre de 2024,"Opinión Pública, Seguridad Social (Salud y Pen..."
1,Perspectivas Económicas,https://espaciopublico.cl/nuestro_trabajo/pers...,4 de noviembre de 2024,Crecimiento Sustentable e Inclusivo
2,Personalización de la política en Chile,https://espaciopublico.cl/nuestro_trabajo/la-d...,21 de octubre de 2024,Democracia y Anticorrupción
3,Instrumentos económicos y financieros para el ...,https://espaciopublico.cl/nuestro_trabajo/inst...,29 de julio de 2024,Sustentabilidad y Recursos Naturales.
4,Evolución de la industria del litio en Chile y...,https://espaciopublico.cl/nuestro_trabajo/evol...,24 de junio de 2024,Sustentabilidad y Recursos Naturales.
...,...,...,...,...
197,«Protección efectiva del consumidor». Document...,https://espaciopublico.cl/nuestro_trabajo/prot...,1 de agosto de 2013,Crecimiento Sustentable e Inclusivo
198,«Opciones de reforma a la seguridad social en ...,https://espaciopublico.cl/nuestro_trabajo/opci...,1 de julio de 2013,Seguridad Social (Salud y Pensiones)
199,«Participación ciudadana en el sector energéti...,https://espaciopublico.cl/nuestro_trabajo/part...,1 de junio de 2013,Sustentabilidad y Recursos Naturales.
200,«Energía y orden territorial». Documento de Re...,https://espaciopublico.cl/nuestro_trabajo/ener...,1 de junio de 2013,Sustentabilidad y Recursos Naturales.


# Prensa

In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def create_driver(headless=False):
    """Crea y devuelve una instancia del driver de Chrome."""
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")  # Recomendado en modo headless
    service = Service()  # Se usa el `Service` para evitar advertencias
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Crear el driver
driver = create_driver(headless=False)

# URL de la página que deseas scrapear
url = "https://espaciopublico.cl/categoria/prensa/"

# Abrir la página
driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(2)  # Ajusta este tiempo si es necesario

# Hacer scroll hasta el final de la página para cargar todo el contenido dinámico
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Desplazarse hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Esperar a que se cargue el contenido nuevo
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # Si no hay más contenido nuevo, salir del bucle
        break
    last_height = new_height

# Obtener el HTML de la página después de cargar todo el contenido
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encontrar el contenedor de los artículos
articulos_container = soup.select_one("#Content > div > div.sections_group > div.section.Array > div > div > div > div.posts_group.lm_wrapper.grid")

# Lista para almacenar los datos
datos = []

# Verificar si el contenedor fue encontrado
if articulos_container:
    # Encontrar todos los artículos dentro del contenedor
    articulos = articulos_container.find_all('div', class_='post-item isotope-item clearfix post type-post status-publish format-standard has-post-thumbnail')
    
    # Iterar sobre cada artículo
    for articulo in articulos:
        # Extraer el título y el enlace
        titulo_tag = articulo.find('h3', class_='posts-entry-title')
        titulo = titulo_tag.text.strip() if titulo_tag else "No title"
        enlace = titulo_tag.find('a')['href'] if titulo_tag and titulo_tag.find('a') else "No link"
        
        # Extraer la fecha
        fecha_tag = articulo.find('div', class_='post-date')
        fecha = fecha_tag.text.strip() if fecha_tag else "No date"
        
        # Extraer el autor
        autor_tag = articulo.find('div', class_='post-authors')
        autor = autor_tag.find('h5').text.strip() if autor_tag and autor_tag.find('h5') else "No author"
        
        # Extraer el extracto
        extracto_tag = articulo.find('div', class_='post-excerpt')
        extracto = extracto_tag.text.strip() if extracto_tag else "No excerpt"
        
        # Guardar los datos en un diccionario
        datos.append({
            "Título": titulo,
            "Enlace": enlace,
            "Fecha": fecha,
            "Autor": autor,
            "Extracto": extracto
        })
else:
    print("No se encontró el contenedor de artículos.")

# Cerrar el navegador
driver.quit()

# Convertir la lista de datos en un DataFrame
df2 = pd.DataFrame(datos)

# Mostrar el DataFrame
df2

# Guardar el DataFrame en un archivo CSV (opcional)
# df.to_csv("articulos_espaciopublico.csv", index=False)

,Título,Enlace,Fecha,Autor,Extracto
0,Reforma de pensiones en su fase decisiva,https://espaciopublico.cl/reforma-de-pensiones...,27 de enero de 2025,Paula Benavides,El camino hacia un acuerdo en materia de pensi...
1,Espacio Público y Horizontal lanzan proyecto «...,https://espaciopublico.cl/espacio-publico-y-ho...,24 de enero de 2025,,Hace más de una década el crecimiento económic...
2,¿Cómo se han insertado jóvenes titulados al me...,https://espaciopublico.cl/como-se-han-insertad...,23 de enero de 2025,"Patricio Domínguez, Martín Latorre",El pasado lunes 8 de enero se dieron a conocer...
3,Avances del acuerdo de pensiones,https://espaciopublico.cl/avances-del-acuerdo-...,22 de enero de 2025,Benjamín García,Las reformas previsionales son difíciles de tr...
4,El acuerdo en pensiones,https://espaciopublico.cl/el-acuerdo-en-pensio...,19 de enero de 2025,Andrea Repetto,Los sistemas de pensiones son complejos. Tiene...
5,Espacio Público analiza retornos de la educaci...,https://espaciopublico.cl/espacio-publico-anal...,16 de enero de 2025,,¿Qué desafíos ha planteado el proceso de expan...
6,¿Quién se beneficia? La urgencia de identifica...,https://espaciopublico.cl/quien-se-beneficia-l...,12 de enero de 2025,Eduardo Engel,"En los últimos años, hemos presenciado una exp..."
7,Sistema político: afinar el diagnóstico,https://espaciopublico.cl/sistema-politico-afi...,8 de enero de 2025,Benjamín García,El proyecto de reforma constitucional presenta...
8,Claridad en acuerdo en pensiones,https://espaciopublico.cl/claridad-en-acuerdo-...,4 de enero de 2025,"Paula Benavides, Patricio Domínguez, José De G...",Señor Director: La necesidad de alcanzar un ac...
9,Viejos y nuevos desafíos en integridad,https://espaciopublico.cl/viejos-y-nuevos-desa...,26 de diciembre de 2024,Benjamín García,El 2024 estuvo marcado por numerosos casos de ...


In [10]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def create_driver(headless=False):
    """Crea y devuelve una instancia del driver de Chrome."""
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")  # Recomendado en modo headless
    service = Service()  # Se usa el `Service` para evitar advertencias
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def scrape_page(driver, url):
    """Scrapea una página y devuelve los datos de los artículos."""
    driver.get(url)
    time.sleep(0.5)  # Esperar a que la página cargue completamente

    # Hacer scroll hasta el final de la página para cargar todo el contenido dinámico
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Esperar a que se cargue el contenido nuevo
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Si no hay más contenido nuevo, salir del bucle
            break
        last_height = new_height

    # Obtener el HTML de la página después de cargar todo el contenido
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar el contenedor de los artículos
    articulos_container = soup.select_one("#Content > div > div.sections_group > div.section.Array > div > div > div > div.posts_group.lm_wrapper.grid")

    # Lista para almacenar los datos
    datos = []

    # Verificar si el contenedor fue encontrado
    if articulos_container:
        # Encontrar todos los artículos dentro del contenedor
        articulos = articulos_container.find_all('div', class_='post-item isotope-item clearfix post type-post status-publish format-standard has-post-thumbnail')
        
        # Iterar sobre cada artículo
        for articulo in articulos:
            # Extraer el título y el enlace
            titulo_tag = articulo.find('h3', class_='posts-entry-title')
            titulo = titulo_tag.text.strip() if titulo_tag else "No title"
            enlace = titulo_tag.find('a')['href'] if titulo_tag and titulo_tag.find('a') else "No link"
            
            # Extraer la fecha
            fecha_tag = articulo.find('div', class_='post-date')
            fecha = fecha_tag.text.strip() if fecha_tag else "No date"
            
            # Extraer el autor
            autor_tag = articulo.find('div', class_='post-authors')
            autor = autor_tag.find('h5').text.strip() if autor_tag and autor_tag.find('h5') else "No author"
            
            # Extraer el extracto
            extracto_tag = articulo.find('div', class_='post-excerpt')
            extracto = extracto_tag.text.strip() if extracto_tag else "No excerpt"
            
            # Guardar los datos en un diccionario
            datos.append({
                "Título": titulo,
                "Enlace": enlace,
                "Fecha": fecha,
                "Autor": autor,
                "Extracto": extracto
            })
    else:
        print(f"No se encontró el contenedor de artículos en la página: {url}")

    return datos

# Crear el driver
driver = create_driver(headless=False)

# Lista para almacenar todos los datos de todas las páginas
todos_los_datos = []

# Iterar sobre las páginas (desde la 1 hasta la 103)
for pagina in range(1, 104):  # range(1, 104) para incluir la página 103
    url = f"https://espaciopublico.cl/categoria/prensa/page/{pagina}/"
    print(f"Scrapeando página: {url}")
    datos_pagina = scrape_page(driver, url)
    todos_los_datos.extend(datos_pagina)

# Cerrar el navegador
driver.quit()

# Convertir la lista de datos en un DataFrame
df2 = pd.DataFrame(todos_los_datos)

# Mostrar el DataFrame
df2

# Guardar el DataFrame en un archivo CSV (opcional)
#df2.to_csv("articulos_espaciopublico_todas_las_paginas.csv", index=False)

Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/1/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/2/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/3/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/4/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/5/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/6/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/7/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/8/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/9/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/10/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/11/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/12/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/13/
Scrapeando página: https://espaciopublico.cl/categoria/prensa/page/14/
Scrapeando pági

,Título,Enlace,Fecha,Autor,Extracto
0,Reforma de pensiones en su fase decisiva,https://espaciopublico.cl/reforma-de-pensiones...,27 de enero de 2025,Paula Benavides,El camino hacia un acuerdo en materia de pensi...
1,Espacio Público y Horizontal lanzan proyecto «...,https://espaciopublico.cl/espacio-publico-y-ho...,24 de enero de 2025,,Hace más de una década el crecimiento económic...
2,¿Cómo se han insertado jóvenes titulados al me...,https://espaciopublico.cl/como-se-han-insertad...,23 de enero de 2025,"Patricio Domínguez, Martín Latorre",El pasado lunes 8 de enero se dieron a conocer...
3,Avances del acuerdo de pensiones,https://espaciopublico.cl/avances-del-acuerdo-...,22 de enero de 2025,Benjamín García,Las reformas previsionales son difíciles de tr...
4,El acuerdo en pensiones,https://espaciopublico.cl/el-acuerdo-en-pensio...,19 de enero de 2025,Andrea Repetto,Los sistemas de pensiones son complejos. Tiene...
...,...,...,...,...,...
2139,Proyecto para fortalecer el Sernac acoge varia...,https://espaciopublico.cl/proyecto-para-fortal...,5 de junio de 2014,,La iniciativa del Ejecutivo le confiere mayore...
2140,Marcela Ríos: “Relación de la ciudadanía con l...,https://espaciopublico.cl/marcela-rios-relacio...,28 de mayo de 2014,,La cientista política y directora de Espacio P...
2141,“El fin de los clubes cerrados: Cómo reducir l...,https://espaciopublico.cl/el-fin-de-los-clubes...,12 de mayo de 2014,,Este lunes 12 de mayo a las 9:00 hrs. en el Ce...
2142,«SEMINARIO SOBRE SEGREGACIÓN ESCOLAR»,https://espaciopublico.cl/seminario-sobre-segr...,19 de diciembre de 2013,,Espacio Público presentó los principales resul...


# Corpus

In [14]:
import pandas as pd

# Lista de DataFrames
dfs = [df, df2]

# Concatenar todos los DataFrames en uno solo
ep = pd.concat(dfs, ignore_index=True)

# Mostrar el DataFrame resultante
ep

,Título,Enlace,Fecha,Áreas de estudio,Autor,Extracto
0,No title,https://espaciopublico.cl/areas_de_estudio/seg...,30 de noviembre de 2024,"Opinión Pública, Seguridad Social (Salud y Pen...",NaN,NaN
1,Perspectivas Económicas,https://espaciopublico.cl/nuestro_trabajo/pers...,4 de noviembre de 2024,Crecimiento Sustentable e Inclusivo,NaN,NaN
2,Personalización de la política en Chile,https://espaciopublico.cl/nuestro_trabajo/la-d...,21 de octubre de 2024,Democracia y Anticorrupción,NaN,NaN
3,Instrumentos económicos y financieros para el ...,https://espaciopublico.cl/nuestro_trabajo/inst...,29 de julio de 2024,Sustentabilidad y Recursos Naturales.,NaN,NaN
4,Evolución de la industria del litio en Chile y...,https://espaciopublico.cl/nuestro_trabajo/evol...,24 de junio de 2024,Sustentabilidad y Recursos Naturales.,NaN,NaN
...,...,...,...,...,...,...
2341,Proyecto para fortalecer el Sernac acoge varia...,https://espaciopublico.cl/proyecto-para-fortal...,5 de junio de 2014,NaN,,La iniciativa del Ejecutivo le confiere mayore...
2342,Marcela Ríos: “Relación de la ciudadanía con l...,https://espaciopublico.cl/marcela-rios-relacio...,28 de mayo de 2014,NaN,,La cientista política y directora de Espacio P...
2343,“El fin de los clubes cerrados: Cómo reducir l...,https://espaciopublico.cl/el-fin-de-los-clubes...,12 de mayo de 2014,NaN,,Este lunes 12 de mayo a las 9:00 hrs. en el Ce...
2344,«SEMINARIO SOBRE SEGREGACIÓN ESCOLAR»,https://espaciopublico.cl/seminario-sobre-segr...,19 de diciembre de 2013,NaN,,Espacio Público presentó los principales resul...


In [15]:
ep.columns

Index(['Título', 'Enlace', 'Fecha', 'Áreas de estudio', 'Autor', 'Extracto'], dtype='object')

In [20]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time

def create_driver(headless=False):
    """Crea y devuelve una instancia del driver de Chrome."""
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")  # Recomendado en modo headless
    service = Service()  # Se usa el `Service` para evitar advertencias
    driver = webdriver.Chrome(service=service, options=options)
    return driver

driver = create_driver(headless=False)

# Cargar el DataFrame
ep2 = ep.copy() 

# Crear la columna "Corpus" si no existe
if 'Corpus' not in ep2.columns:
    ep2['Corpus'] = None
    ep2["Tipo"] = None

# Función para extraer el corpus
def extract_corpus(url):
    driver.get(url)
    time.sleep(1)  # Esperar a que la página cargue

    try:
        # Intentar extraer el corpus del primer formato
        corpus_container = driver.find_element(By.CSS_SELECTOR, "#Content > div > div.sections_group > div > div.section.mcb-section.mcb-section-3dd2c9463.content-sidebar > div > div.wrap.mcb-wrap.mcb-wrap-2cfe5bd5f.one.valign-top.clearfix > div > div.column.mcb-column.mcb-item-xsufmzocp.one.column_column > div")
        paragraphs = corpus_container.find_elements(By.TAG_NAME, "p")
        corpus = "\n".join([p.text for p in paragraphs])
        return corpus
    except NoSuchElementException:
        try:
            # Intentar extraer el corpus del segundo formato (ficha-proyectos)
            corpus_container = driver.find_element(By.CSS_SELECTOR, "div.ficha-proyectos h5")
            paragraphs = corpus_container.find_elements(By.TAG_NAME, "p")
            corpus = "\n".join([p.text for p in paragraphs])
            return corpus
        except NoSuchElementException:
            return None

# Función para extraer título, áreas de estudio, autor y tipo
def extract_attributes(url):
    driver.get(url)
    time.sleep(1)  # Esperar a que la página cargue

    attributes = {
        'Título': None,
        'Áreas de estudio': None,
        'Autor': None,
        'Tipo': None
    }

    try:
        # Intentar extraer el título del primer formato
        title_element = driver.find_element(By.CSS_SELECTOR, "#Content > div > div.sections_group > div > div.section.mcb-section.mcb-section-3dd2c9463.content-sidebar > div > div.wrap.mcb-wrap.mcb-wrap-a64fdbaf3.three-fifth.seccion-areas.valign-top.clearfix > div > div.column.mcb-column.mcb-item-1a0633259.one.column_column.column-margin-0px > div > h3")
        attributes['Título'] = title_element.text
    except NoSuchElementException:
        try:
            # Intentar extraer el título del segundo formato (ficha-proyectos)
            title_element = driver.find_element(By.CSS_SELECTOR, "div.ficha-proyectos h3")
            attributes['Título'] = title_element.text
        except NoSuchElementException:
            pass

    try:
        # Intentar extraer áreas de estudio del primer formato
        areas_element = driver.find_element(By.CSS_SELECTOR, "#Content > div > div.sections_group > div > div.section.mcb-section.mcb-section-3dd2c9463.content-sidebar > div > div.wrap.mcb-wrap.mcb-wrap-a64fdbaf3.three-fifth.seccion-areas.valign-top.clearfix > div > div.column.mcb-column.mcb-item-1a0633259.one.column_column.column-margin-0px > div > span.area-post")
        attributes['Áreas de estudio'] = areas_element.text
    except NoSuchElementException:
        try:
            # Intentar extraer áreas de estudio del segundo formato (ficha-proyectos)
            areas_element = driver.find_element(By.CSS_SELECTOR, "span.ficha-proyecto-area")
            attributes['Áreas de estudio'] = areas_element.text.replace("Área de estudio: ", "")
        except NoSuchElementException:
            pass

    try:
        # Intentar extraer autor del primer formato
        author_element = driver.find_element(By.CSS_SELECTOR, "#Content > div > div.sections_group > div > div.section.mcb-section.mcb-section-3dd2c9463.content-sidebar > div > div.wrap.mcb-wrap.mcb-wrap-a64fdbaf3.three-fifth.seccion-areas.valign-top.clearfix > div > div.column.mcb-column.mcb-item-1a0633259.one.column_column.column-margin-0px > div > div.column_attr.clearfix.post-ficha > span.autores-post")
        attributes['Autor'] = author_element.text
    except NoSuchElementException:
        try:
            # Intentar extraer autor del segundo formato (ficha-proyectos)
            author_elements = driver.find_elements(By.CSS_SELECTOR, "span.ficha-proyecto-equipo ul li a")
            authors = [author.text for author in author_elements]
            attributes['Autor'] = ", ".join(authors)
        except NoSuchElementException:
            pass

    try:
        # Intentar extraer tipo del segundo formato (ficha-proyectos)
        tipo_element = driver.find_element(By.CSS_SELECTOR, "span.ficha-proyecto-tipo")
        attributes['Tipo'] = tipo_element.text.replace("Tipo: ", "")
    except NoSuchElementException:
        pass

    return attributes

# Iterar sobre cada fila del DataFrame
for index, row in ep2.iterrows():
    url = row['Enlace']
    
    # Extraer el corpus
    corpus = extract_corpus(url)
    if corpus:
        ep2.at[index, 'Corpus'] = corpus
    
    # Extraer y actualizar atributos si están vacíos o son NaN
    attributes = extract_attributes(url)
    for key, value in attributes.items():
        if pd.isna(row[key]) or row[key] == 'No title':
            ep2.at[index, key] = value

# Guardar el DataFrame actualizado
# ep2.to_csv('ep_actualizado.csv', index=False)

# Cerrar el navegador
driver.quit()

ep2

,Título,Enlace,Fecha,Áreas de estudio,Autor,Extracto,Corpus,Tipo
0,None,https://espaciopublico.cl/areas_de_estudio/seg...,30 de noviembre de 2024,"Opinión Pública, Seguridad Social (Salud y Pen...",,NaN,None,None
1,Perspectivas Económicas,https://espaciopublico.cl/nuestro_trabajo/pers...,4 de noviembre de 2024,Crecimiento Sustentable e Inclusivo,"Patricio Domínguez, Paula Benavides, Eduardo B...",NaN,La discusión por el desarrollo económico y los...,Tipo:\nPropuestas
2,Personalización de la política en Chile,https://espaciopublico.cl/nuestro_trabajo/la-d...,21 de octubre de 2024,Democracia y Anticorrupción,"Javier Sajuria, Julieta Suárez-Cao, Matías Piña",NaN,"En los últimos años, la ciudadanía se ha acost...",Tipo:\nPropuestas
3,Instrumentos económicos y financieros para el ...,https://espaciopublico.cl/nuestro_trabajo/inst...,29 de julio de 2024,Sustentabilidad y Recursos Naturales.,,NaN,Con el objetivo de aportar en la elaboración d...,Tipo:\nMinuta técnica
4,Evolución de la industria del litio en Chile y...,https://espaciopublico.cl/nuestro_trabajo/evol...,24 de junio de 2024,Sustentabilidad y Recursos Naturales.,,NaN,La industria mundial del litio experimentó un ...,Tipo:\nMinuta técnica
...,...,...,...,...,...,...,...,...
2341,Proyecto para fortalecer el Sernac acoge varia...,https://espaciopublico.cl/proyecto-para-fortal...,5 de junio de 2014,Crecimiento Sustentable e Inclusivo,,La iniciativa del Ejecutivo le confiere mayore...,"\nEl proyecto para fortalecer el Sernac, envia...",None
2342,Marcela Ríos: “Relación de la ciudadanía con l...,https://espaciopublico.cl/marcela-rios-relacio...,28 de mayo de 2014,Democracia y Anticorrupción,,La cientista política y directora de Espacio P...,\nEn medio del dinámico debate público que se ...,None
2343,“El fin de los clubes cerrados: Cómo reducir l...,https://espaciopublico.cl/el-fin-de-los-clubes...,12 de mayo de 2014,"Jóvenes, Educación y Empleo",,Este lunes 12 de mayo a las 9:00 hrs. en el Ce...,\nBajo el objetivo de hacer propuestas para fo...,None
2344,«SEMINARIO SOBRE SEGREGACIÓN ESCOLAR»,https://espaciopublico.cl/seminario-sobre-segr...,19 de diciembre de 2013,"Jóvenes, Educación y Empleo",,Espacio Público presentó los principales resul...,\nEspacio Público presentó los principales res...,None


In [21]:
ep2.to_excel("Espacio_publico.xlsx", index=False)